In [1]:
import numpy as np
import gym
from gym import spaces, logger
from gym.utils import seeding
import copy
import or_gym

In [155]:
# Matches the AWS model as first pass

class OnlineBinPackingEnv(gym.Env):
    '''
    Online Bin Packing Problem

    The Bin Packing Problem (BPP) is a combinatorial optimization problem which
    requires the user to select from a range of goods of different values and
    weights in order to maximize the value of the selected items within a 
    given weight limit. This version is online meaning each item is randonly
    presented to the algorithm one at a time, at which point the algorithm 
    can either accept or reject the item. After seeing a fixed number of 
    items are shown, the episode terminates. If the weight limit is reached
    before the episode ends, then it terminates early.

    Observation:
        Type: Tuple, Discrete
        0 - bin_capacity: Count of bins at a given level h
        -1: Current item size


    Actions:
        Type: Discrete
        0: Open a new bin and place item into bin
        1+: Attempt to place item into bin at the given level

    Reward:
        Negative of the waste, which is the difference between the current
        size and excess space of the bin.

    Starting State:
        No available bins and random starting item
        
    Episode Termination:
        When invalid action is selected (e.g. attempt to place item in non-existent
        bin), bin limits are exceeded, or step limit is reached.
    '''
    def __init__(self):
        self.bin_capacity = 9
        self.item_sizes = [2, 3]
        self.item_probs = [0.8, 0.2]
        self.step_counter = 0
        self.step_limit = 1000
        
        self.observation_space = spaces.Box(
            low=np.array([0] * (1 + self.bin_capacity)),
            high=np.array([self.step_limit] * self.bin_capacity + [max(self.item_sizes)]),
            dtype=np.uint32)
        
        self.action_space = spaces.Discrete(self.bin_capacity)
        
        self.seed()
        self.state = self.reset()
        
    def step(self, action):
        done = False
        if action >= self.bin_capacity:
            raise ValueError('{} is an invalid action. Must be between {} and {}'.format(
                action, 0, self.bin_capacity))
        elif action > (self.bin_capacity - self.item_size):
            # Bin overflows
            reward = BIG_NEG_REWARD - self.waste
            done = True
        elif action == 0:
            # Create new bin
            self.bin_levels[self.item_size] += 1
            # This waste penalty seems very strange, it only occurs
            # when a new bin is opened.
            self.waste = self.bin_capacity - self.item_size
            reward = -1 * self.waste
        elif self.bin_levels[action] == 0:
            # Can't insert item into non-existent bin
            reward = BIG_NEG_REWARD - self.waste
            done = True
        else:
            if action + self.item_size == self.bin_capacity:
                self.num_full_bins += 1
            else:
                self.bin_levels[action + self.item_size] += 1
            self.waste = -self.item_size
            reward = -1 * self.waste
            
            self.bin_levels[action] -= 1
        
        self.total_reward += reward
        
        if self.step_counter >= self.step_limit:
            done = True
            
        if self.step_counter == self.step_limit:
            done = True
            
        self.item_size = self.get_item()
        state = self.bin_levels + [self.item_size]
        
        return self.state, reward, done, {}
    
    def get_item(self):
        return np.random.choice(self.item_sizes, p=self.item_probs)
        
    def sample_action(self):
        return self.action_space.sample()
    
    def reset(self):
        self.current_weight = 0
        self.step_counter = 0        
        self.num_full_bins = 0
        self.total_reward = 0
        self.waste = 0
        self.step_counter = 0
        
        self.bin_levels = [0] * self.bin_capacity
        self.item_size = self.get_item()
        initial_state = self.bin_levels + [self.item_size]
        return initial_state

# Online Bin Packing

A random sequence of items $i$ with size $w_i \in W_i$, is provided to an algorithm to be packed into various bins $B$ denoted by $j \in J$, where the capacity of each bin is given by $\textrm{max}(W_i) \leq B_j^{max} < \infty \; \forall j \in J$. Each item $i$ arrives at a given time $t \in T$, and is sampled according to a fixed probability $p_i$. Following Gupta and Radovanovic, we assume all bin and item sizes are integer values.

## Amazon Implementation

Unlimited bins, each with identical capacity. Assign to group of bins based on level $h$, whereby $N_h(t)$ is the number of bins at that level. The action, $a_t$ is to select the level to assign the item to, or, in the case of $a_t=0$, to open a new, empty bin. $R_t$ is the negative of the incremental waste as $i$ is assigned to $B_j$. They use an action mask to prevent infeasible actions.

They proivde a large negative reward (-100) for any actions that prematurely end the episode. Additionally, the reward function seems odd as implemented by the AWS team. In the [original paper](https://arxiv.org/pdf/1211.2687.pdf), it is given as:

$$W_F(n) = \sum_{h=1}^{B-1} N_h(n)(B-h)$$

where $W_F(n)$ is the total accumulated empty space in partially filled bins after n items from the distribution $F$ have been packed by the algorithm. The AWS implementation returns $B-h$ for every new bin created, a positive reward if it does fit in an existing bin, 

In [7]:
env_config = {'mask': True}
env = or_gym.make('BinPacking-v0', env_config=env_config)

In [8]:
env.state

{'state': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3]),
 'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0.])}

In [9]:
env.step(0)

({'state': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 2]),
  'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'action_mask': array([1., 0., 0., 1., 0., 0., 0., 0., 0.])},
 -6,
 False,
 {})

In [10]:
env.step(3)

({'state': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 2]),
  'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'action_mask': array([1., 0., 0., 0., 0., 1., 0., 0., 0.])},
 2,
 False,
 {})

In [11]:
env.step(5)

({'state': array([0, 0, 0, 0, 0, 0, 0, 1, 0, 2]),
  'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'action_mask': array([1., 0., 0., 0., 0., 0., 0., 1., 0.])},
 2,
 False,
 {})

In [12]:
env.step(7)

({'state': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2]),
  'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'action_mask': array([1., 0., 0., 0., 0., 0., 0., 0., 0.])},
 2,
 False,
 {})

In [20]:
env.step(7)

({'state': array([0, 0, 0, 0, 0, 0, 1, 1, 0, 2]),
  'avail_actions': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  'action_mask': array([1., 0., 0., 0., 0., 0., 1., 1., 0.])},
 -98,
 True,
 {})

In [21]:
x = np.arange(10)
a, b, c, d, e = x[:5]

In [22]:
a, b, c, d, e


(0, 1, 2, 3, 4)

In [23]:
a

0

In [25]:
x = np.zeros(5)
for i in range(5):
    d = np.random.poisson(5)
    print(d)
    x_new = np.zeros(x.shape)
    x_new[:-1] += x[1:]
    x_new[-1] = d
    x = x_new.copy()
    
x

3
7
7
1
5


array([3., 7., 7., 1., 5.])

In [26]:
np.hstack([np.zeros(2), x])

array([0., 0., 3., 7., 7., 1., 5.])

In [27]:
np.array_split(np.array(range(-5, 6)), 4)

[array([-5, -4, -3]), array([-2, -1,  0]), array([1, 2, 3]), array([4, 5])]

In [32]:
np.vstack([(0, 1)] * 2).flatten()

array([0, 1, 0, 1])

In [33]:
grid = (5, 5)

In [34]:
loc_permutations = [[x, y] for y in range(grid[1]) 
                    for x in range(grid[0])]
loc_permutations

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (0, 2),
 (1, 2),
 (2, 2),
 (3, 2),
 (4, 2),
 (0, 3),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 3),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (4, 4)]

In [44]:
import random

In [59]:
dr = list(random.sample(loc_permutations, 1)[0])

In [60]:
dr

[0, 0]

In [57]:
if np.random.random(1) < 9:
    print('x')

x


In [62]:
random.sample(loc_permutations, 3, 

TypeError: sample() got an unexpected keyword argument 'replace'

In [66]:
l = [1, 2, 3, 4, 5, 6]
random.sample(l, 7)

ValueError: Sample larger than population or is negative